<a href="https://colab.research.google.com/github/Nightmare125/Cyberbullying-Detection/blob/main/Cyberbullying_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Preprocessing

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from google.colab import drive

Load Drive

In [11]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load Dataset

In [12]:
data_path = "/content/drive/MyDrive/Cyberbullying Dataset/"
train_df = pd.read_csv(data_path + "cyberbullying_tweets Training.csv")
val_df = pd.read_csv(data_path + "cyberbullying_tweets Validating.csv")
test_df = pd.read_csv(data_path + "cyberbullying_tweets Testing.csv")
collective_df = pd.read_csv(data_path + "cyberbullying_tweets.csv")

Load the pre-trained tokenizer for text processing

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_text(text):
    """Tokenize and preprocess input text."""
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

Custom dataset class for Cyberbullying detection

In [14]:
class CyberbullyingDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = [preprocess_text(text) for text in texts]
        self.labels = labels.tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {key: val.squeeze(0) for key, val in self.encodings[idx].items()}, torch.tensor(self.labels[idx])

Dataset Objects for Training, Validation and Testing

In [21]:
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
val_df.columns = val_df.columns.str.strip()

In [22]:
print(train_df.columns)  # Check the available columns
print(test_df.columns)
print(val_df.columns)

Index(['tweet_text', 'cyberbullying_type'], dtype='object')
Index(['tweet_text', 'cyberbullying_type'], dtype='object')
Index(['tweet_text', 'cyberbullying_type'], dtype='object')


In [23]:
train_dataset = CyberbullyingDataset(train_df["tweet_text"], train_df["cyberbullying_type"])
test_dataset = CyberbullyingDataset(test_df["tweet_text"], test_df["cyberbullying_type"])
val_dataset = CyberbullyingDataset(val_df["tweet_text"], val_df["cyberbullying_type"])

Apply Preprocessing

In [ ]:
df["tweet_text"] = df["tweet_text"].apply(preprocess_text)

Split Data

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df["tweet_text"], df["label"], test_size=0.2, random_state=42)

Load Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenization function

In [ ]:
def tokenize_text(texts):
  return tokenizer(list(texts), padding=True, truncation=True, max_length=128, return_tensors="pt")

train_encodings = tokenize_text(train_texts)
val_encodings = tokenize_text(val_texts)

Convert labels to Tensor

In [ ]:
train_labels = torch.tensor(train_labels.tolist())
val_labels = torch.tensor(val_labels.tolist())

### Training the Model

In [ ]:
class CyberbullyingDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
      return len(self.labels)

    def __getitem__(self, idx):
      return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}, torch.tensor(self.labels[idx])


Create Dataset Objects

In [ ]:
train_dataset=
test_dataset=
val_dataset=

SyntaxError: invalid syntax (<ipython-input-23-3952010849b9>, line 1)

In [ ]:
train_dataset = CyberbullyingDataset(train_encodings, train_labels)
val_dataset = CyberbullyingDataset(val_encodings, val_labels)

Data Loaders

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

Load Pretrained model for classification

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Optimizer and Loss

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training loop

In [ ]:
epochs = 3
train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in range(epochs):
  model.train()
  train_preds, train_truths, train_loss = [], [], 0.0
  for batch in train_loader:
    inputs, labels = batch
    inputs = {key: val.to(device) for key, val in inputs.items()}
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(**inputs)
    loss = loss_fn(outputs.logits, labels)
    loss.backward()
    optimizer.step()

    train_loss +=loss.item()
    train_preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
    train_truths.extend(labels.cpu().numpy())

  train_acc = accuracy_score(train_truths, train_preds)
  train_losses.append(train_loss / len(train_loader))
  train_accuracies.append(train_acc)

  model.eval()
  val_preds, val_truths, val_loss = [], [], 0.0
  with torch.no_grad():
    for batch in val_loader:
      inputs, labels = batch
      inputs = {key: val.to(device) for key, val in inputs.items()}
      labels = labels.to(device)

      outputs = model(**inputs)
      loss = loss_fn(outputs.logits, labels)
      val_loss += loss.item()

      val_preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
      val_truths.extend(labels.cpu().numpy())

  val_acc= accuracy_score(val_truths, val_preds)
  val_losses.append(val_loss / len(val_loader))
  val_accuracies.append(val_acc)

  print(f"Epoch {epoch+1}: Train Acc= {train_acc:.4f}, Val Acc= {val_acc:.4f}, Train Loss= {train_losses[-1]:.4f}, Val Loss={val_losses[-1]:.4f}")


<ipython-input-12-f1f27cedbb67>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}, torch.tensor(self.labels[idx])


Plot Accuracy and Loss Graphs

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_accuracies, label='Train Accuracy', marker='o')
plt.plot(range(1, epochs + 1), val_accuracies, label='Val Accuracy', marker='s')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, epochs + 1), val_losses, label='Val Loss', marker='s')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()

plt.show()

Train Model

Test model

Validate model